In [1]:
# Script completo: exportar ventas por mes y año y monitorear consumo de placas con alertas semanales
# Requerimiento previo: instala psycopg2 con:
#     pip install psycopg2-binary
import pandas as pd
import psycopg2
import smtplib
from email.mime.text import MIMEText
from datetime import datetime, timedelta

# --- VARIABLES DE ENTRADA: MES Y AÑO DE INTERÉS Y RUTA DE EXPORTACIÓN ---
meses_interes = [8,9,10,11,12,1,2,3,4,5]
años_interes = [2024,2025]
ruta_exportacion = (
    r"C:\Users\Miriam Martinez\OneDrive - Natgas\Planeación comercial\ANÁLISIS\GNC\ventas_"
    + "_".join(map(str, años_interes))
    + "_"
    + "_".join(map(str, meses_interes))
    + ".xlsx"
)

# --- PLACAS GNC A MONITOREAR ---
NPLATES = [
    '51UD2U','54US8S','57UG7X','60UG7X','GENDISTA','GENENCO',
    'GENSIMSA','NEOMEXICANA','E5773','E6713','7HU2382','E5772',
    'E6712','E5771','GENCIMAGAS','55AX8N','537XT1','GENSIMA',
    '08UH8N','80UH8N','03AN2H','80UH8P','70UD2S','GEMSIMSA',
    'GSIMSA','97UH6P','17UJ4J','18AU6X','18UG6X','7HU7608',
    '7HU7607','07UC7G','80UH8','86UR6H','02UR6M','02UC3X',
    '03UR6M','51UR8M','551XA4','555XA4','71HU7C','71UH7C',
    '0711UC7G','07UC76','71UH7','GENGREEN','35UD1K','550XA4',
    '551XA','8HU6323','04UR6M','63UW1H','65UW1H','78UP7G',
    '79UP7G','80UP7G','81UP7G','98UC2X'
]

# --- FUNCIONES AUXILIARES ---
def obtener_ventas_mensuales(conn, meses, años, placas):
    meses_str = ",".join(map(str, meses))
    años_str = ",".join(map(str, años))
    query = f"""
        SELECT placa, SUM(cantidad) AS suma_cantidad,
               EXTRACT(YEAR FROM fecha) AS año,
               EXTRACT(MONTH FROM fecha) AS mes
        FROM erelis2_ventas_total
        WHERE EXTRACT(MONTH FROM fecha) IN ({meses_str})
          AND EXTRACT(YEAR FROM fecha) IN ({años_str})
          AND placa = ANY(%s)
        GROUP BY placa, año, mes
        ORDER BY placa, año, mes;
    """
    df = pd.read_sql(query, conn, params=(placas,))
    df['mes'] = df['mes'].astype(int)
    df['año'] = df['año'].astype(int)
    df['mes_año'] = df['mes'].astype(str) + '_' + df['año'].astype(str)
    return df


def enviar_alerta(placa, mensaje):
    # Configuración fija de correo
    remitente = 'mmartinez@natgas.com.mx'
    destinatarios = ['mmartinez@natgas.com.mx']
    password = 'Lunes22'

    msg = MIMEText(mensaje)
    msg['Subject'] = f'Alerta consumo placa {placa}'
    msg['From'] = remitente
    msg['To'] = ','.join(destinatarios)

    with smtplib.SMTP('smtp.natgas.com', 587) as servidor:
        servidor.starttls()
        servidor.login(remitente, password)
        servidor.send_message(msg)

# --- CONEXIÓN A POSTGRES ---
conn = psycopg2.connect(
    dbname="postgres",
    user="erelis_admin",
    password="WQyS2HkgE7jRSi",
    host="erelis-prod.postgres.database.azure.com",
    port=5432
)

# --- EXPORTACIÓN DE DATOS MENSUALES ---
df_mensual = obtener_ventas_mensuales(conn, meses_interes, años_interes, NPLATES)
df_mensual.to_excel(ruta_exportacion, index=False)
print(f"Archivo exportado en: {ruta_exportacion}")

# --- ALERTAS POR VARIACIÓN SEMANAL ---
# Calcular rangos semanales
hoy = datetime.now().date()
domingo = hoy - timedelta(days=hoy.weekday()) + timedelta(days=6)
lunes = domingo - timedelta(days=6)
prev_domingo = domingo - timedelta(days=7)
prev_lunes = prev_domingo - timedelta(days=6)

# Consultas semanales usando pandas
df_actual = pd.read_sql(
    """
    SELECT placa, SUM(cantidad) AS total_litros
    FROM erelis2_ventas_total
    WHERE fecha >= %s AND fecha < %s AND placa = ANY(%s)
    GROUP BY placa;
    """, conn, params=(lunes, domingo + timedelta(days=1), NPLATES)
)
df_previa = pd.read_sql(
    """
    SELECT placa, SUM(cantidad) AS total_litros
    FROM erelis2_ventas_total
    WHERE fecha >= %s AND fecha < %s AND placa = ANY(%s)
    GROUP BY placa;
    """, conn, params=(prev_lunes, prev_domingo + timedelta(days=1), NPLATES)
)
conn.close()

# Convertir resultados a diccionarios
dict_act = dict(zip(df_actual['placa'], df_actual['total_litros']))
dict_prev = dict(zip(df_previa['placa'], df_previa['total_litros']))

# Envío de alertas semanales
for placa in NPLATES:
    act = dict_act.get(placa, 0)
    prev = dict_prev.get(placa, 0)
    if act == 0:
        enviar_alerta(placa, f"Sin consumo para placa {placa} en la semana del {lunes} al {domingo}.")
    elif prev > 0 and act < prev:
        diff = prev - act
        pct = (diff / prev) * 100
        enviar_alerta(
            placa,
            f"Consumo de placa {placa} bajó de {prev:.0f}L a {act:.0f}L ({pct:.1f}% menos) "
            f"en la semana del {lunes} al {domingo}."
        )



C:\Users\Miriam Martinez\AppData\Local\Temp\ipykernel_22536\2975622616.py:50: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, params=(placas,))


Archivo exportado en: C:\Users\Miriam Martinez\OneDrive - Natgas\Planeación comercial\ANÁLISIS\GNC\ventas_2024_2025_8_9_10_11_12_1_2_3_4_5.xlsx


C:\Users\Miriam Martinez\AppData\Local\Temp\ipykernel_22536\2975622616.py:96: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_actual = pd.read_sql(
C:\Users\Miriam Martinez\AppData\Local\Temp\ipykernel_22536\2975622616.py:104: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_previa = pd.read_sql(


gaierror: [Errno 11001] getaddrinfo failed

In [ ]:
pip install psycopg2-binary

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ------------------------------------ --- 1.0/1.2 MB 12.7 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 7.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
